In [1]:
import os
import re

from dotenv import load_dotenv
from langchain import hub
from langchain.agents import AgentType, load_tools
from langchain_experimental.tools import PythonREPLTool
from langchain.agents.initialize import initialize_agent
from langchain.agents import create_react_agent, AgentExecutor
from langchain.agents.format_scratchpad import format_log_to_str
from langchain.agents.output_parsers import JSONAgentOutputParser
from langchain.chains.conversation.memory import ConversationBufferWindowMemory
from langchain_openai import ChatOpenAI
from langchain.tools import ArxivQueryRun, WikipediaQueryRun
from langchain.tools.render import render_text_description_and_args
from langchain.utilities import ArxivAPIWrapper, WikipediaAPIWrapper
from langchain.prompts import MessagesPlaceholder

from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.callbacks.base import BaseCallbackManager

from llamp.mp.agents import (
    MPSummaryExpert,
    MPThermoExpert,
    MPElasticityExpert,
    MPDielectricExpert,
    MPPiezoelectricExpert,
    MPMagnetismExpert,
    MPElectronicExpert,
    MPSynthesisExpert,
    MPStructureRetriever,
)
# from llamp.arxiv.agents import ArxivAgent

load_dotenv()

MP_API_KEY = os.getenv("MP_API_KEY", None)

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY", None)
OPENAI_ORGANIZATION = os.getenv("OPENAI_ORGANIZATION", None)


OPENAI_GPT_MODEL = "gpt-4-1106-preview"
# OPENAI_GPT_MODEL = "gpt-4-0125-preview"
# OPENAI_GPT_MODEL = "gpt-3.5-turbo-1106"
# OPENAI_GPT_MODEL = "gpt-4-turbo"

In [2]:
top_llm = ChatOpenAI(
    temperature=0.1,
    model=OPENAI_GPT_MODEL,
    openai_api_key=OPENAI_API_KEY,
    openai_organization=OPENAI_ORGANIZATION,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()],
)

bottom_callback_handler = StreamingStdOutCallbackHandler()

bottom_llm = ChatOpenAI(
    temperature=0,
    model=OPENAI_GPT_MODEL,
    openai_api_key=OPENAI_API_KEY,
    openai_organization=OPENAI_ORGANIZATION,
    max_retries=5,
    streaming=True,
    callbacks=[bottom_callback_handler],
)


wikipedia = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())
arxiv = ArxivQueryRun(api_wrapper=ArxivAPIWrapper())

tools = load_tools(["llm-math"], llm=bottom_llm)
tools += [PythonREPLTool()]
tools += [
    MPThermoExpert(llm=bottom_llm, mp_api_key=MP_API_KEY).as_tool(
        agent_kwargs=dict(return_intermediate_steps=False)
    ),
    MPElasticityExpert(llm=bottom_llm, mp_api_key=MP_API_KEY).as_tool(
        agent_kwargs=dict(return_intermediate_steps=False)
    ),
    MPDielectricExpert(llm=bottom_llm, mp_api_key=MP_API_KEY).as_tool(
        agent_kwargs=dict(return_intermediate_steps=False)
    ),
    MPMagnetismExpert(llm=bottom_llm, mp_api_key=MP_API_KEY).as_tool(
        agent_kwargs=dict(return_intermediate_steps=False)
    ),
    MPElectronicExpert(llm=bottom_llm, mp_api_key=MP_API_KEY).as_tool(
        agent_kwargs=dict(return_intermediate_steps=False)
    ),
    MPPiezoelectricExpert(llm=bottom_llm, mp_api_key=MP_API_KEY).as_tool(
        agent_kwargs=dict(return_intermediate_steps=False)
    ),
    MPSummaryExpert(llm=bottom_llm, mp_api_key=MP_API_KEY).as_tool(
        agent_kwargs=dict(return_intermediate_steps=False)
    ),
    MPSynthesisExpert(llm=bottom_llm, mp_api_key=MP_API_KEY).as_tool(
        agent_kwargs=dict(return_intermediate_steps=False)
    ),
    MPStructureRetriever(llm=bottom_llm, mp_api_key=MP_API_KEY).as_tool(
        agent_kwargs=dict(return_intermediate_steps=False)
    ),
    arxiv,
    wikipedia,
]

instructions = re.sub(
        r"\s+",
        " ",
        """You are a data-aware agent that can consult materials-related
    data through Materials Project (MP) database, arXiv, Wikipedia, and a python 
    REPL, which you can use to execute python code. If you get an error, debug 
    your code and try again. Only use the output of your code to answer the 
    question. Ask user to clarify their queries if needed. Please note that you 
    don't have direct control over MP but through multiple assistant agents to 
    help you. You need to provide complete context in the input for assistants to 
    do their job. REFINE and CRITQUE the output of the assistants if needed. You
    can end the conversation by saying "Bye".
    """,
    ).replace("\n", " ")
base_prompt = hub.pull("langchain-ai/react-agent-template")
prompt = base_prompt.partial(instructions=instructions)

conversational_memory = ConversationBufferWindowMemory(
    memory_key="chat_history", k=5, return_messages=True
)

agent = create_react_agent(top_llm, tools, prompt)
agent_executor = AgentExecutor(
    agent=agent, tools=tools, verbose=True,
    handle_parsing_errors=True,
    memory=conversational_memory,
)

In [3]:

pattern = re.compile(r'\b' + re.escape("Bye") + r'\b[.!?]*')

while True:
    query = input("Query:")
    print(f"Query: {query}")
    response = agent_executor.invoke(
        {
            "input": query,
        }
    )
    if re.search(pattern, response["output"]):
        break

Query: Fetch the stable Au from MP and use ASE calculator to relax it


> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: MPSummaryExpert
Action Input: stable AuThought: Do I need to use a tool? Yes
Action: MPSummaryExpert
Action Input: stable Au

> Entering new AgentExecutor chain...
Action:
```
{
  "action": "search_materials_summary__get",
  "action_input": {
    "formula": "Au",
    "is_stable": true
  }
}
```Action:
```
{
  "action": "search_materials_summary__get",
  "action_input": {
    "formula": "Au",
    "is_stable": true
  }
}
```Error on search_materials_summary__get: `fields` must be specified in the query. Please provide more detailed request arguments or try smaller request by specifying 'limit' in request.Action:
```
{
  "action": "search_materials_summary__get",
  "action_input": {
    "formula": "Au",
    "is_stable": true,
    "fields": "material_id,formula_pretty,composition,nsites,symmetry"
  }
}
```Action:
```
{
  "action": "sear

Retrieving SummaryDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

[{'nsites': 1, 'composition': {'Au': 1.0}, 'formula_pretty': 'Au', 'symmetry': {'crystal_system': 'Cubic', 'symbol': 'Fm-3m', 'number': 225, 'point_group': 'm-3m', 'symprec': 0.1, 'version': '2.0.2'}, 'material_id': 'mp-81'}]Action:
```
{
  "action": "Final Answer",
  "action_input": "The stable form of gold (Au) on the Materials Project database has the material ID mp-81. It has a cubic crystal system with the space group Fm-3m (number 225) and a point group of m-3m. The structure consists of a single site (nsites: 1) occupied by gold."
}
```Action:
```
{
  "action": "Final Answer",
  "action_input": "The stable form of gold (Au) on the Materials Project database has the material ID mp-81. It has a cubic crystal system with the space group Fm-3m (number 225) and a point group of m-3m. The structure consists of a single site (nsites: 1) occupied by gold."
}
```

> Finished chain.
{'input': 'stable Au', 'output': 'The stable form of gold (Au) on the Materials Project database has the ma

Retrieving SummaryDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

[{'material_id': 'mp-81', 'structure': {'@module': 'pymatgen.core.structure', '@class': 'Structure', 'charge': 0, 'lattice': {'matrix': [[2.554382, 0.0, 1.474773], [0.851461, 2.408295, 1.474773], [0.0, 0.0, 2.949546]], 'pbc': [True, True, True], 'a': 2.9495462029697044, 'b': 2.9495467521426066, 'c': 2.949546, 'alpha': 60.00000843542354, 'beta': 60.000002276344645, 'gamma': 60.00000120154803, 'volume': 18.14473805188449}, 'properties': {}, 'sites': [{'species': [{'element': 'Au', 'occu': 1}], 'abc': [0.0, 0.0, 0.0], 'xyz': [0.0, 0.0, 0.0], 'properties': {'magmom': -0.0}, 'label': 'Au'}]}}]


> Finished chain.
{'input': 'mp-81', 'output': [{'material_id': 'mp-81', 'structure': {'@module': 'pymatgen.core.structure', '@class': 'Structure', 'charge': 0, 'lattice': {'matrix': [[2.554382, 0.0, 1.474773], [0.851461, 2.408295, 1.474773], [0.0, 0.0, 2.949546]], 'pbc': [True, True, True], 'a': 2.9495462029697044, 'b': 2.9495467521426066, 'c': 2.949546, 'alpha': 60.00000843542354, 'beta': 60.00000

Retrieving SummaryDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

[{'material_id': 'mp-81', 'structure': {'@module': 'pymatgen.core.structure', '@class': 'Structure', 'charge': 0, 'lattice': {'matrix': [[2.554382, 0.0, 1.474773], [0.851461, 2.408295, 1.474773], [0.0, 0.0, 2.949546]], 'pbc': [True, True, True], 'a': 2.9495462029697044, 'b': 2.9495467521426066, 'c': 2.949546, 'alpha': 60.00000843542354, 'beta': 60.000002276344645, 'gamma': 60.00000120154803, 'volume': 18.14473805188449}, 'properties': {}, 'sites': [{'species': [{'element': 'Au', 'occu': 1}], 'abc': [0.0, 0.0, 0.0], 'xyz': [0.0, 0.0, 0.0], 'properties': {'magmom': -0.0}, 'label': 'Au'}]}}]


> Finished chain.
{'input': 'mp-81', 'output': [{'material_id': 'mp-81', 'structure': {'@module': 'pymatgen.core.structure', '@class': 'Structure', 'charge': 0, 'lattice': {'matrix': [[2.554382, 0.0, 1.474773], [0.851461, 2.408295, 1.474773], [0.0, 0.0, 2.949546]], 'pbc': [True, True, True], 'a': 2.9495462029697044, 'b': 2.9495467521426066, 'c': 2.949546, 'alpha': 60.00000843542354, 'beta': 60.00000

Python REPL can execute arbitrary code. Use with caution.


I need to use a tool to execute Python code using ASE's EMT calculator to relax the structure retrieved from the Materials Project.
Action: Python_REPL
Action Input: 
```python
from ase import Atoms
from ase.calculators.emt import EMT
from ase.optimize import BFGS
from pymatgen.core.structure import Structure

# Define the structure from the Materials Project JSON output
lattice_matrix = [[2.554382, 0.0, 1.474773], [0.851461, 2.408295, 1.474773], [0.0, 0.0, 2.949546]]
positions = [[0.0, 0.0, 0.0]]
symbols = ['Au']

# Create the ASE Atoms object
atoms = Atoms(symbols=symbols, positions=positions, cell=lattice_matrix, pbc=True)

# Set up the EMT calculator
atoms.set_calculator(EMT())

# Relax the structure
optimizer = BFGS(atoms)
optimizer.run(fmax=0.01)

# Get the relaxed structure information
relaxed_positions = atoms.get_positions()
relaxed_cell = atoms.get_cell()

print(relaxed_positions, relaxed_cell)
```      Step     Time          Energy          fmax
BFGS:    0 10:19:00        0.

AuthenticationError: Error code: 401 - {'error': {'message': 'Incorrect API key provided: sk-XZjGa***************************************jK7R. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}

In [ ]:
response